<a href="https://colab.research.google.com/github/duygusezr/magaza_yorumlari/blob/main/magaza_yorumlari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import re
from tensorflow.keras.callbacks import EarlyStopping
import chardet

In [2]:
# Dosyanın yolunu belirleyelim
file_path = "magaza_yorumlari.csv"

# Dosyanın encoding türünü belirleyelim
with open(file_path, "rb") as f:
    raw_data = f.read(100000)  # İlk 100KB'ı oku
    result = chardet.detect(raw_data)

# Encoding türünü alalım
encoding_type = result["encoding"]
print(f"Dosyanın encoding türü: {encoding_type}")

# Dosyayı belirlenen encoding ile tekrar okuyalım
df = pd.read_csv(file_path, encoding=encoding_type)

# İlk birkaç satırı görüntüleyelim
print(df.head())

Dosyanın encoding türü: UTF-16
                                               Görüş    Durum
0  Ürün gayet güzel boyutları  dört kişilik aile ...   Olumlu
1                     Ürünü kullandım gayet memnunum   Olumlu
2                                        Hızlı kargo   Olumlu
3  8 yıl önce lg 42 lm serisi almıştım ve tv den ...  Olumsuz
4  ürün fena değil ancak gövdenin plastik olması ...   Olumlu


In [3]:
# Metni temizleme fonksiyonu
def clean_text(text):
    text = str(text).lower()  # NaN değerler varsa stringe çevir
    text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldır
    return text

# Metni temizleyelim
df["Görüş"] = df["Görüş"].apply(clean_text)

# Tokenizer ile kelimeleri sayısal değerlere çevirme
tokenizer = Tokenizer(num_words=4000)  # En sık kullanılan 4000 kelimeyi al
tokenizer.fit_on_texts(df["Görüş"])
sequences = tokenizer.texts_to_sequences(df["Görüş"])

# Maksimum uzunluk belirleme ve pad sequences uygulama
max_length = 40
X = pad_sequences(sequences, maxlen=max_length)

# Etiketleri one-hot encoding formatına çevirme
label_mapping = {label: idx for idx, label in enumerate(df["Durum"].unique())}
df["Durum_Num"] = df["Durum"].map(label_mapping)
y = to_categorical(df["Durum_Num"], num_classes=len(label_mapping))

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sonuçları kontrol edelim
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6792, 40) (1699, 40) (6792, 2) (1699, 2)


In [4]:
# LSTM Modeli oluşturma
vocab_size = 4000  # Tokenizer'daki kelime sayısı
embedding_dim = 100  # Her kelime için vektör boyutu

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(128, return_sequences=True))  # İlk LSTM katmanı
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))  # İkinci LSTM katmanı
model.add(Dropout(0.2))
model.add(LSTM(32))  # Üçüncü LSTM katmanı
model.add(Dropout(0.2))
model.add(Dense(len(label_mapping), activation='softmax'))

# Modeli derleme
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modeli özetleyelim
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Daha uzun bekleme süresi
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.6729 - loss: 0.5673 - val_accuracy: 0.8982 - val_loss: 0.2766
Epoch 2/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9243 - loss: 0.2113 - val_accuracy: 0.9070 - val_loss: 0.2596
Epoch 3/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9582 - loss: 0.1293 - val_accuracy: 0.8982 - val_loss: 0.2768
Epoch 4/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9719 - loss: 0.0906 - val_accuracy: 0.8988 - val_loss: 0.3170
Epoch 5/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9771 - loss: 0.0809 - val_accuracy: 0.8935 - val_loss: 0.3810
Epoch 6/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9797 - loss: 0.0775 - val_accuracy: 0.8911 - val_loss: 0.4115
Epoch 7/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9837 - loss: 0.0525 - val_accuracy: 0.8846 - val_loss: 0.4577
Epoch 8/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9880 - loss: 0.0399 - val_acc

In [6]:
# Metin tahmini fonksiyonu
def predict_sentiment(text):
    cleaned_text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    prediction = model.predict(padded_sequence)
    sentiment = np.argmax(prediction)
    return sentiment  # 0: olumsuz, 1: olumlu, 2: nötr

# Örnek tahmin
test_text = "iyi paketlenmiş"
sentiment_result = predict_sentiment(test_text)
if sentiment_result == 0:
    print("Olumlu")
elif sentiment_result == 1:
    print("Olumsuz")
else:
    print("Nötr")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
Olumlu
